In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)  
cap.set(3, wCam)
cap.set(4, hCam)

screen_width, screen_height = pyautogui.size()

with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    pTime = 0
    prev_x, prev_y = None, None 

    while True:
        success, img = cap.read()
        if not success:
            break
        
        img = cv2.flip(img, 1)

        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_img)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                x1, y1 = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x, hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
                x2, y2 = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x, hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
                h, w, _ = img.shape
                x1, y1 = int(x1 * w), int(y1 * h)
                x2, y2 = int(x2 * w), int(y2 * h)

                length = np.hypot(x2 - x1, y2 - y1)

                if length < 40:  
                    pyautogui.click()  
                    cv2.putText(img, "Fist Detected - Left Click", (40, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)

                cursor_x = np.interp(x1, [0, w], [0, screen_width])
                cursor_y = np.interp(y1, [0, h], [0, screen_height])

                if prev_x is not None and prev_y is not None:
                    pyautogui.moveTo(cursor_x, cursor_y)

                prev_x, prev_y = cursor_x, cursor_y

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Gesture Cursor Control with Fist Click", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()